<img align="right" src="../img/auvisus.svg" width="100" >


# DEtection TRansformer Network

---

## 1. Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from detr_models.detr.model import DETR
from detr_models.detr.train import get_image_information

from detr_models.data_feeder.pvoc_feeder import PVOCFeeder
from detr_models.data_feeder.coco_feeder import COCOFeeder
from detr_models.data_feeder.uuid_iterator import UUIDIterator
from detr_models.detr.config import DefaultDETRConfig
from detr_models.backbone.backbone import Backbone

import tensorflow as tf

import os

---

## 2. Initialize Model

In [ ]:
# Specify storage path
storage_path = input(prompt='Please specify the storage path:\n')

In [ ]:
# Additional Information used for initialization
config = DefaultDETRConfig()

input_shape, count_images = get_image_information(storage_path, config.data_type)

backbone_config = {
        "input_shape": input_shape,
        "include_top": False,
        "weights": "imagenet",
    }

# Init. model
detr = DETR(
        input_shape=input_shape,
        num_queries=config.num_queries,
        num_classes=config.num_classes,
        num_heads=config.num_heads,
        dim_transformer=config.dim_transformer,
        dim_feedforward=config.dim_feedforward,
        num_transformer_layer=config.num_transformer_layer,
        backbone_name=config.backbone_name,
        backbone_config=backbone_config,
        train_backbone=False,
)


---

## 3. Build Model

In [ ]:
# Build Model
model = detr.build_model()
model.summary()

---

## 4. Train Model

### 4.1. Initialize Training Helper

In [ ]:
# Init Data Feeder PVOC type
pvoc_feeder = PVOCFeeder(
    storage_path=storage_path,
    num_queries=config.num_queries,
    num_classes=config.num_classes,
    batch_size=config.batch_size,
)

In [ ]:
# Init Data Feeder COCO type
coco_feeder = COCOFeeder(
    storage_path=storage_path,
    num_queries=config.num_queries,
    num_classes=config.num_classes,
    batch_size=config.batch_size,
    image_width=config.image_width,
    image_height=config.image_height
)

In [ ]:
if config.data_type == "PVOC":
    print("Use Input Data in PascalVOC format")
    data_feeder = pvoc_feeder
else:
    print("Use Input Data in COCO format")
    data_feeder = coco_feeder

**You have to choose which DataFeeder type you gonna use for training**

---

### 4.2. Specify Training Configuration

In [ ]:
# Init Optimizer
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
epochs = 30
output_dir = "./saved_model"

# Specify the path to any pre-trained model if you want to use one
use_pretrained = False

training_config = {
        "epochs" : epochs,
        "optimizer" : optimizer,
        "count_images" : count_images,
        "use_pretrained" : use_pretrained,
        "output_dir" : output_dir,
        "batch_size" : config.batch_size,
        "verbose" : False
    }

### 4.3. Start Training

In [ ]:
detr.train(
    training_config=training_config,
    optimizer=optimizer,
    count_images=count_images,
    data_feeder=data_feeder,
)

---

*This Notebook was created by: [auvisus GmbH](https://www.auvisus.com/)*